In [ ]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
#data = pd.read_csv('../input/kddcup.data_10_percent_corrected')
#data
labels = pd.read_csv("labels.csv",sep = ":",header = None)
data   = pd.read_csv("kddcup.data_10_percent_corrected", names = labels.iloc[:,0].values)
data_onehotencoded = data.copy()

In [ ]:
lb_make = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)
cat_columns = labels.loc[labels.iloc[:,1] == " symbolic.",0].values
data[cat_columns] = data[cat_columns].apply(lambda col: lb_make.fit_transform(col))

In [ ]:
replacements = {
        r'(smurf.|neptune.|back.|teardrop.|pod.|land.)' : 'dos',
        r'(normal.)' : 'normal',
        r'(satan.|ipsweep.|portsweep.|nmap.)' : 'probe',
        r'(warezclient.|guess_passwd.|warezmaster.|imap.|ftp_write.|multihop.|phf.|spy.)' : 'r2l',
        r'(buffer_overflow.|rootkit.|loadmodule.|perl.)' : 'u2r'
    }
data.replace(replacements, regex=True, inplace=True)
print(data.iloc[:,41].unique())

In [ ]:
def ffr(df, t):
    classes = ['normal', 'u2r', 'dos', 'r2l', 'probe']
    df_new = df.iloc[:,:-1]
    var_d = []
    for clas in classes:
        temp = df.loc[df['output'] == clas]
        #print("is unique ", temp.output.unique())
        temp = temp.iloc[:,:-1].values
        #print("temp = ", temp)
        min_max_scaler = preprocessing.MinMaxScaler()
        temp_scaled = min_max_scaler.fit_transform(temp)
        #print("temp_scaled = ", temp_scaled)
        mean_ind = temp_scaled.mean(axis = 0)
        #print("mean_ind = ", mean_ind)
        var_d_f = np.square(mean_ind - temp).mean(axis = 0)
        #print("var_d_f_f = ", var_d_f)
        var_d.append(var_d_f)
    var_d = np.array(var_d)
    #print(var_d)
    var_d_mean = var_d.mean(axis = 0)
    
    
#     var = np.zeros(len(mean_means))
#     for i in means:
#         var += np.square(i - mean_means)
#     var /= len(mean_means)
#     #print(var)
    indexes = list(np.argsort(var_d_mean))
    filtered_indexes = indexes[:t]
    filtered_indexes.append(41)
    filtered_data = df.iloc[:, filtered_indexes]
    return filtered_data

In [ ]:
data = ffr(data, 10)

In [ ]:
# Separating Data and the Output labels into data and Y respectively

print(data.shape)
Y = data.iloc[:,-1]
Y = np.array(Y)
#Y = Y.reshape(Y.shape[0],1)
print(Y.shape)
data.drop(data.columns[-1], axis=1, inplace=True)
print(data.shape)

In [ ]:
# Splitting Data into train and test
from sklearn.model_selection import train_test_split  
X_train, X_test, Y_train, Y_test = train_test_split(data, Y, test_size = 0.20) 

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
#Y_train = Y_train.reshape(Y_train.shape[0],1)
Y_test = np.array(Y_test)
#Y_test = Y_test.reshape(Y_test.shape[0],1)

print("X_train : ",X_train.shape)
print("X_test : ",X_test.shape)
print("Y_train : ",Y_train.shape)
print("Y_test : ",Y_test.shape)

In [ ]:
# Preprocessing in order to do the computation faster doing scaling can use MinMaxScaler as well...
# Check how SVM learns in terms of distance calculation

X_train = X_train.astype(float)
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
print("X_train Shape : ", X_train.shape)

Y_train = np.array(Y_train)
encoder = LabelEncoder()
encoder.fit(Y_train)
Y_train = encoder.transform(Y_train)
print("Y_train Shape : ",Y_train.shape)

X_test = X_test.astype(float)
scaler = preprocessing.StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)
print("X_train Shape : ", X_train.shape)

Y_test = encoder.transform(Y_test)
print("Y_test Shape : ",Y_test.shape)

In [ ]:

#flatten_Y_train = Y_train.ravel()
#class_weights = class_weight.compute_class_weight('balanced' ,np.unique(Y_train) ,flatten_Y_train)
# grid_param = {
#     'kernel': ['rbf','linear'],
#     'C' : [1,10,100],
#     'gamma':['auto','scale']
# }

# svm_model = GridSearchCV(SVC(class_weight='balanced'),param_grid=grid_param,cv=5,n_jobs=-1,verbose = 1)
# print("Done Setting Up Model..")
# svm_model.fit(X_train, Y_train)
# print("Done Fitting..")


svm_model = svm.SVC(kernel='rbf',C=100)
svm_model.fit(X_train,Y_train)
print("Done Fitting..")

In [ ]:
# print('Best score for training data:', svm_model.best_score_,"\n") 

# # View the best parameters for the model found using grid search
# print('Best C:',svm_model.best_estimator_.C,"\n") 
# print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
# #print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

In [ ]:
#final_model = svm_model.best_estimator_
Y_pred = svm_model.predict(X_test)
print(confusion_matrix(Y_test,Y_pred))
print("\n")
print(classification_report(Y_test,Y_pred))

In [ ]:
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import precision_recall_fscore_support
# predicted = predicted.astype(int)
# conf = confusion_matrix(Y_test,predicted)
# print(conf)

# precision,recall,f1,support = precision_recall_fscore_support(Y_test,predicted)
# print("Precision : ",precision)
# print("Recall : ",recall)
# print("f1 : ",f1)
# print("Support : ",support)

In [ ]:
pred = Y_pred.tolist()
print("0 occurence in Predicted List : ",pred.count(0))
print("1 occurence in Predicted List : ",pred.count(1))
print("2 occurence in Predicted List : ",pred.count(2))
print("3 occurence in Predicted List : ",pred.count(3))
print("4 occurence in Predicted List : ",pred.count(4))

In [ ]:
actual = Y_test.tolist()
print("0 occurence in Actual List : ",actual.count(0))
print("1 occurence in Actual List : ",actual.count(1))
print("2 occurence in Actual List : ",actual.count(2))
print("3 occurence in Actual List : ",actual.count(3))
print("4 occurence in Actual List : ",actual.count(4))